Para evaluar el desempeño de un modelo **ARIMA** o **ARIMAX** en un proyecto que busca predecir casos de dengue con datos meteorológicos en Caucasia, es clave usar métricas que midan **precisión predictiva, capacidad explicativa y validez estadística del modelo**. Estas se agrupan en tres niveles: ajuste del modelo, error de predicción y diagnóstico de residuos.

---

# 1) Métricas de error de predicción (las más importantes en tu caso)

Como el objetivo es **predecir el número de casos de dengue**, estas métricas evalúan qué tan cerca están las predicciones de los valores reales.

## a) MAE — Error absoluto medio

[
MAE = \frac{1}{n}\sum_{t=1}^{n} |y_t - \hat{y}_t|
]

* Fácil de interpretar.
* Mide el error promedio en unidades reales (casos de dengue).
* Robusto ante valores extremos.

**Interpretación en tu proyecto:**
“En promedio, el modelo se equivoca en X casos por periodo.”

---

## b) RMSE — Raíz del error cuadrático medio

[
RMSE = \sqrt{\frac{1}{n}\sum_{t=1}^{n}(y_t-\hat{y}_t)^2}
]

* Penaliza errores grandes.
* Muy útil cuando errores altos son críticos (picos epidémicos).

**Recomendado para epidemiología.**

---

## c) MAPE — Error porcentual absoluto medio

[
MAPE = \frac{100}{n}\sum_{t=1}^{n}\left|\frac{y_t-\hat{y}_t}{y_t}\right|
]

* Interpretable como porcentaje.
* Problema: falla cuando hay valores cercanos a 0.

---

## d) sMAPE — versión robusta del MAPE

Más estable para series con valores pequeños.

---

# 2) Métricas de ajuste del modelo (comparación ARIMA vs ARIMAX)

Sirven para decidir cuál modelo describe mejor la serie temporal.

## a) AIC — Akaike Information Criterion

[
AIC = 2k - 2\ln(L)
]

* Penaliza modelos complejos.
* **Menor AIC = mejor modelo.**

Muy útil para comparar:

* ARIMA(p,d,q)
* ARIMAX(p,d,q) con variables meteorológicas

---

## b) BIC — Bayesian Information Criterion

[
BIC = k\ln(n) - 2\ln(L)
]

* Penaliza aún más la complejidad.
* Favorece modelos más simples.

---

# 3) Diagnóstico de residuos (validez estadística)

Un buen ARIMA/ARIMAX debe dejar residuos como **ruido blanco**.

## a) Prueba de Ljung-Box

Evalúa autocorrelación en residuos.

* H₀: los residuos son independientes.
* Si p > 0.05 → modelo adecuado.

---

## b) Autocorrelation Function (ACF) de residuos

Debe mostrar valores cercanos a cero.

---

## c) Normalidad de residuos

* Shapiro-Wilk
* Jarque-Bera

Importante para intervalos de predicción epidemiológicos.

---

# 4) Métricas específicas para modelos ARIMAX

En ARIMAX se agregan variables meteorológicas:

* temperatura
* precipitación
* humedad
* radiación UV

Aquí evaluamos además:

## a) Significancia de coeficientes

[
p < 0.05
]

Indica que la variable meteorológica explica el dengue.

---

## b) Impacto causal interpretativo

Ejemplo:

* aumento de lluvia → aumento de casos
* rezagos de 2–8 semanas → coherencia epidemiológica

---

# 5) Validación temporal (clave en tu investigación)

No se debe usar validación aleatoria.

Usar:

### a) Train / test temporal

* Entrenar: 2015–2022
* Probar: 2023–2024

### b) Rolling forecast

Entrenamiento progresivo.

Esto simula predicción real en salud pública.

---

# 6) Métricas recomendadas para tu proyecto de dengue

Prioriza:

### Núcleo epidemiológico

* RMSE
* MAE
* MAPE / sMAPE

### Selección del modelo

* AIC
* BIC

### Validación estadística

* Ljung-Box
* ACF de residuos

### Interpretación científica

* significancia de variables climáticas

---

# 7) Ejemplo de flujo metodológico

1. Modelar ARIMA base (solo casos dengue)
2. Modelar ARIMAX (con clima)
3. Comparar:

   * AIC
   * RMSE
4. Validar residuos
5. Evaluar interpretabilidad epidemiológica

---

# 8) Métrica más importante según el objetivo

Depende del propósito del proyecto:

### Predicción operativa en salud pública

→ RMSE + MAE

### Publicación científica

→ AIC + BIC + Ljung-Box

### Comprensión del fenómeno dengue-clima

→ ARIMAX + significancia de coeficientes

---

# 9) Recomendación metodológica fuerte

Para dengue, la literatura muestra que:

* ARIMA predice tendencia
* ARIMAX explica causalidad climática

Por tanto:

**La combinación ideal es:**

* ARIMAX + validación temporal + RMSE.

---

# 10) Siguiente paso investigativo

Puedo ayudarte a construir:

* pipeline metodológico completo
* script en Python (statsmodels)
* diseño de sección metodológica para artículo
* marco de indicadores epidemiológicos

Solo dime cuál necesitas:

1. Código Python
2. Diseño metodológico de investigación
3. Justificación teórica ARIMA vs ARIMAX
4. Interpretación epidemiológica del clima en dengue
5. Todo el pipeline completo para tesis o artículo

Y lo armamos directamente sobre tu dataset meteorológico.


Excelente, Marco. Vamos a construir un **ejemplo completo y didáctico de ARIMAX** aplicado al contexto de predicción de casos de dengue usando variables meteorológicas simuladas, inspirado en tu proyecto en Caucasia.

La idea es que puedas usar esto directamente en un cuaderno Jupyter con tus estudiantes o como base metodológica para artículo.

---

# 🌡️ 1️⃣ Simulación del escenario epidemiológico

Supongamos:

* Serie mensual (10 años → 120 meses)
* Variables meteorológicas:

  * Temperatura media
  * Precipitación
* El dengue depende:

  * De su pasado (componente AR)
  * De la lluvia con rezago
  * De la temperatura actual

---

# 📊 2️⃣ Generación de datos simulados

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)

n = 120
fechas = pd.date_range(start="2015-01-01", periods=n, freq="M")

# Variables meteorológicas simuladas
temperatura = 28 + 2*np.sin(np.linspace(0, 12*np.pi, n)) + np.random.normal(0,0.5,n)
lluvia = 200 + 50*np.sin(np.linspace(0, 6*np.pi, n)) + np.random.normal(0,20,n)

# Componente AR(1) para dengue
errores = np.random.normal(0,5,n)
casos = np.zeros(n)

for t in range(1,n):
    casos[t] = (
        0.6*casos[t-1] +      # efecto autoregresivo
        0.03*lluvia[t-1] +    # lluvia con rezago
        1.2*temperatura[t] +  # temperatura actual
        errores[t]
    )

data = pd.DataFrame({
    "casos_dengue": casos,
    "temperatura": temperatura,
    "lluvia": lluvia
}, index=fechas)

data.plot(subplots=True, figsize=(10,8))
plt.tight_layout()
plt.show()
```

---

# 🔍 3️⃣ Ajustar modelo ARIMA base (sin clima)

```python
from statsmodels.tsa.arima.model import ARIMA

modelo_arima = ARIMA(data["casos_dengue"], order=(1,0,0))
resultado_arima = modelo_arima.fit()

print("AIC ARIMA:", resultado_arima.aic)
```

---

# 🌦️ 4️⃣ Ajustar modelo ARIMAX (con variables meteorológicas)

```python
modelo_arimax = ARIMA(
    data["casos_dengue"],
    order=(1,0,0),
    exog=data[["temperatura","lluvia"]]
)

resultado_arimax = modelo_arimax.fit()

print("AIC ARIMAX:", resultado_arimax.aic)
```

---

# 📌 5️⃣ Comparación de modelos

```python
print("Comparación AIC")
print("----------------")
print("ARIMA :", resultado_arima.aic)
print("ARIMAX:", resultado_arimax.aic)
```

👉 El modelo con menor AIC es preferible.

En esta simulación, el ARIMAX debería tener menor AIC porque los datos fueron generados usando clima.

---

# 📈 6️⃣ Interpretación de coeficientes

```python
print(resultado_arimax.summary())
```

Debes observar:

* Coeficiente AR(1) significativo
* Temperatura significativa
* Lluvia significativa

Si los p-valores < 0.05 → variables climáticas influyen estadísticamente.

---

# 🧪 7️⃣ Diagnóstico de residuos

```python
resultado_arimax.plot_diagnostics(figsize=(10,8))
plt.show()
```

Revisar:

* Residuos tipo ruido blanco
* ACF sin autocorrelaciones fuertes

---

# 📊 8️⃣ Evaluación predictiva (Train-Test temporal)

```python
train = data.iloc[:100]
test = data.iloc[100:]

modelo = ARIMA(
    train["casos_dengue"],
    order=(1,0,0),
    exog=train[["temperatura","lluvia"]]
)

resultado = modelo.fit()

predicciones = resultado.forecast(
    steps=len(test),
    exog=test[["temperatura","lluvia"]]
)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test["casos_dengue"], predicciones))

print("RMSE:", rmse)
```

---

# 🎯 9️⃣ Interpretación epidemiológica

Si obtienes:

* ARIMAX con menor AIC
* Variables significativas
* RMSE bajo
* Residuos sin autocorrelación

Entonces puedes afirmar:

> Existe evidencia estadística de que las variables meteorológicas mejoran la predicción de dengue en Caucasia.

---

# 🧠 10️⃣ Diferencia conceptual ARIMA vs ARIMAX

| Modelo | Qué explica                            |
| ------ | -------------------------------------- |
| ARIMA  | Dinámica interna de la enfermedad      |
| ARIMAX | Dinámica interna + factores climáticos |

En epidemiología tropical, el ARIMAX suele capturar mejor el fenómeno porque el mosquito Aedes depende fuertemente de condiciones climáticas.

---

# 📘 11️⃣ Cómo usar esto en tu investigación

Puedes estructurar tu sección metodológica así:

1. Modelado ARIMA base
2. Inclusión de variables meteorológicas
3. Comparación por AIC
4. Validación temporal
5. Interpretación epidemiológica

---

Si quieres, puedo ahora:

* 🔬 Construirte la versión lista para artículo científico
* 📊 Agregar rezagos climáticos óptimos automáticos
* 📈 Incluir validación rolling forecast avanzada
* 🧮 Derivar matemáticamente el modelo ARIMAX
* 🧠 Conectarlo con SVM y Random Forest para comparación

¿Lo llevamos a nivel artículo Q1 o a nivel clase avanzada para tus estudiantes?
